In [1]:
import pandas as pd
import cv2
import numpy as np

# Load the CSV file
file_path = "C1_352_121.csv"  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Extract X and Y coordinates
x = df["Drillhole.X"].values
y = df["Drillhole.Y"].values

# Normalize coordinates for plotting (adjust to fit within the canvas size)
canvas_size = (1000, 1000)  # Width x Height
margin = 50  # Margin around the points
min_x, max_x = x.min(), x.max()
min_y, max_y = y.min(), y.max()

# Scale coordinates to fit within the canvas
scaled_x = ((x - min_x) / (max_x - min_x)) * (canvas_size[0] - 2 * margin) + margin
scaled_y = ((y - min_y) / (max_y - min_y)) * (canvas_size[1] - 2 * margin) + margin

# Create a blank canvas with a transparent background
canvas = np.zeros((canvas_size[1], canvas_size[0], 4), dtype=np.uint8)

# Draw points on the canvas
for sx, sy in zip(scaled_x, scaled_y):
    cv2.circle(canvas, (int(sx), int(sy)), radius=6, color=(0, 255, 0, 255), thickness=-1)  # Green dots with full opacity

# Save the drill hole map as a PNG with transparency
output_path = "drill_hole_map_cv2.png"
cv2.imwrite(output_path, canvas)

# Add the scaled pixel coordinates directly to the existing DataFrame
df["Pixel_X"] = scaled_x.astype(int)
df["Pixel_Y"] = scaled_y.astype(int)

# Save the updated DataFrame with the pixel coordinates
output_csv_path = "drillholes_with_pixels_coordinates.csv"  # Replace with your desired output file path
df.to_csv(output_csv_path, index=False)

print(f"Updated DataFrame with pixel coordinates saved to {output_csv_path}")

In [16]:
import cv2
import numpy as np


# Load images
design_map = cv2.imread("drill_hole_map_cv2.png")
drone_image = cv2.imread("demo1_frames/demo1_frame_0000.jpg")

# Function to manually select points
def select_points(image, window_name):
    points = []
    def mouse_click(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            print(f"Point selected: ({x}, {y})")
    
    cv2.imshow(window_name, image)
    cv2.setMouseCallback(window_name, mouse_click)
    print(f"Select 4 points in the {window_name} window...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return points

# Select points from the design map and drone image
design_points = select_points(design_map, "Design Map")
drone_points = select_points(drone_image, "Drone Footage")

# Convert points to numpy arrays
design_points = np.array(design_points, dtype=np.float32)
drone_points = np.array(drone_points, dtype=np.float32)

# Calculate homography matrix
H, status = cv2.findHomography(design_points, drone_points)

np.save("homography_matrix.npy", H)
print("Homography matrix saved to 'homography_matrix.npy'")


Select 4 points in the Design Map window...
Point selected: (2255, 972)
Point selected: (799, 1800)
Point selected: (134, 666)
Point selected: (961, 120)
Select 4 points in the Drone Footage window...
Point selected: (396, 497)
Point selected: (1186, 457)
Point selected: (1336, 609)
Point selected: (648, 641)
Homography matrix saved to 'homography_matrix.npy'


In [ ]:
import cv2
import numpy as np

# Load the design map (with transparency) and drone image (JPEG format)
design_map = cv2.imread("drill_hole_map_cv2.png", cv2.IMREAD_UNCHANGED)  # Load with transparency (RGBA)
drone_image = cv2.imread("demo1_frames/demo1_frame_0000.jpg")  # JPEG format

# Convert the drone image to RGBA
drone_image = cv2.cvtColor(drone_image, cv2.COLOR_BGR2BGRA)

# Load the homography matrix
H = np.load("homography_matrix.npy")

# Warp the design map to align with the drone footage
aligned_map = cv2.warpPerspective(design_map, H, (drone_image.shape[1], drone_image.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

# Blend the aligned design map onto the drone image
# Normalize alpha channel of the aligned map
alpha_map = aligned_map[:, :, 3] / 255.0  # Normalized alpha values (0.0 to 1.0)

# Blend each channel (R, G, B)
for c in range(3):  # Iterate over the R, G, B channels
    drone_image[:, :, c] = (
        alpha_map * aligned_map[:, :, c] + (1 - alpha_map) * drone_image[:, :, c]
    ).astype(np.uint8)

# Combine the alpha channels: keep the maximum alpha for each pixel
drone_image[:, :, 3] = np.maximum(drone_image[:, :, 3], aligned_map[:, :, 3])

# Save and display the final result
cv2.imwrite("overlay_result.png", drone_image)

True

: 